# Part 2: Basic LLM Chat Tool

## Introduction

In this part, you'll create a simple command-line chat tool that interacts with a Large Language Model (LLM) through the Hugging Face API. This tool will allow you to have conversations with an LLM about healthcare topics.

## Learning Objectives

- Connect to the Hugging Face API
- Create a basic interactive chat loop
- Handle simple error cases
- Test with healthcare questions

## Setup and Installation

In [1]:
# Import necessary libraries
import os
import sys
import requests
import time
import logging
import argparse
from typing import Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create directories
os.makedirs('utils', exist_ok=True)
os.makedirs('results/part_2', exist_ok=True)

## 1. Connecting to the Hugging Face API

The Hugging Face Inference API provides access to many language models. We'll use models that are available on the free tier.

In [2]:
# Example of a simple API request to Hugging Face
# API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-base" # cannot find this model
API_URL = "https://router.huggingface.co/cohere/compatibility/v1/chat/completions"
from keys import HF_API_KEY  # Import your Hugging Face API key
headers = {"Authorization": f"Bearer {HF_API_KEY}"}  # Optional for some models

def query(payload):
    """
    Send a query to the Hugging Face API
    
    Args:
        payload: Dictionary containing the query parameters
        
    Returns:
        The API response
    """
    # TODO: Implement the API request
    # Use requests.post to send the query to the API_URL
    # Return the response

    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        logger.error(f"Error: {response.status_code} - {response.text}")
        return None
    pass

In [3]:
# Test the query function
test_payload = {
    "messages": [
        {
            "role": "user",
            "content": "What are the symptoms of diabetes?"
        }
    ],
    "model": "command-r-plus-04-2024"
}
# {"inputs": "What are the symptoms of diabetes?"}
response = query(test_payload)
print(response['choices'][0]['message']['content'] if response else "No response received.")

Diabetes is a chronic condition that affects the way the body regulates blood sugar, or glucose. There are two main types of diabetes: type 1 and type 2. The symptoms of diabetes can vary depending on the type and the individual, but some common symptoms include:

- Frequent urination: People with diabetes may urinate more often than usual, as the body tries to get rid of the excess glucose in the blood.
- Increased thirst: As the body tries to replace the fluid lost through frequent urination, people with diabetes may feel more thirsty than usual.
- Hunger: The body may not be able to get enough energy from the food eaten, leading to increased hunger.
- Weight loss: Despite eating more, people with diabetes may lose weight as the body breaks down muscle and fat for energy.
- Fatigue: The body may feel tired and weak as it is not getting enough energy from food.
- Blurred vision: High blood sugar levels can cause the lenses of the eyes to swell, leading to blurred vision.
- Slow-healin

## 2. Creating Simple Chat Scripts

Your task is to create two simple scripts that interact with the Hugging Face API:

1. A basic one-off chat script (`utils/one_off_chat.py`)
2. A contextual conversation script (`utils/conversation.py`)

### One-Off Chat Script

Create a script that handles independent interactions (each prompt/response is separate):

In [4]:
%%writefile utils/one_off_chat.py
# utils/one_off_chat.py

Overwriting utils/one_off_chat.py


In [15]:
import argparse

def get_response(prompt, model_name="command-r-plus-04-2024", api_key=None):
    """
    Get a response from the model
    
    Args:
        prompt: The prompt to send to the model
        model_name: Name of the model to use
        api_key: API key for authentication (optional for some models)
        
    Returns:
        The model's response
    """
    # TODO: Implement the get_response function
    # Set up the API URL and headers
    # Create a payload with the prompt
    # Send the payload to the API
    # Extract and return the generated text from the response
    # Handle any errors that might occur

    payload = {
        'messages': [
            {'role': 'user', 'content': prompt}
        ],
        'model': model_name
    }
    response = query(payload)
    if response and 'choices' in response:
        return response['choices'][0]['message']['content']
    else:
        logger.error("Failed to get a valid response from the model.")
        return None
    pass

def run_chat():
    """Run an interactive chat session"""
    print("Welcome to the Simple LLM Chat! Type 'exit' to quit.")
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
            
        # TODO: Get response from the model
        # Print the response
        response = get_response(user_input)
        if response:
            print(f"Model: {response}")
        else:
            print("Model: Sorry, I couldn't process that.")
        
def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM")
    # TODO: Add arguments to the parser
    parser.add_argument('--model', type=str, default='command-r-plus-04-2024', help='Model name to use for chat')
    parser.add_argument('--prompt', type=str, help='Initial prompt to send to the model')
    args = parser.parse_args()
    model = args.model
    prompt = args.prompt
    response = get_response(prompt, model_name=model) if prompt else None
    if response:
        print(f"Model: {response}")
    else:
        print("Model: Sorry, I couldn't process that.")
    
    
    # TODO: Run the chat function with parsed arguments
    
if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--model MODEL] [--prompt PROMPT]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/macbook/Library/Jupyter/runtime/kernel-v3373b6908a90486a9d10f5e5c57d1ac89c7666edf.json


SystemExit: 2

/Users/macbook/Documents/ucsf_couses/ds223/assignments/7-transformers-Exynos-8890/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Contextual Conversation Script

Create a script that maintains conversation history:

In [ ]:
%%writefile utils/conversation.py
# utils/conversation.py

In [ ]:
def get_response(prompt, history=None, model_name="google/flan-t5-base", api_key=None, history_length=3):
    """
    Get a response from the model using conversation history
    
    Args:
        prompt: The current user prompt
        history: List of previous (prompt, response) tuples
        model_name: Name of the model to use
        api_key: API key for authentication
        history_length: Number of previous exchanges to include in context
        
    Returns:
        The model's response
    """
    # TODO: Implement the contextual response function
    # Initialize history if None
    if history is None:
        history = []
        
    # TODO: Format a prompt that includes previous exchanges
    # Get a response from the API
    # Return the response
    pass

def run_chat():
    """Run an interactive chat session with context"""
    print("Welcome to the Contextual LLM Chat! Type 'exit' to quit.")
    
    # Initialize conversation history
    history = []
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
            
        # TODO: Get response using conversation history
        # Update history
        # Print the response
        
def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM using conversation history")
    # TODO: Add arguments to the parser
    
    args = parser.parse_args()
    
    # TODO: Run the chat function with parsed arguments
    
if __name__ == "__main__":
    main()

In [ ]:
%%writefile utils/conversation.py
# utils/conversation.py

import requests
import argparse
import os

def get_response(prompt, history=None, model_name="google/flan-t5-base", api_key=None, history_length=3):
    """
    Get a response from the model using conversation history
    
    Args:
        prompt: The current user prompt
        history: List of previous (prompt, response) tuples
        model_name: Name of the model to use
        api_key: API key for authentication
        history_length: Number of previous exchanges to include in context
        
    Returns:
        The model's response
    """
    # TODO: Implement the contextual response function
    # Initialize history if None
    if history is None:
        history = []
        
    # TODO: Format a prompt that includes previous exchanges
    # Get a response from the API
    # Return the response
    pass

def run_chat():
    """Run an interactive chat session with context"""
    print("Welcome to the Contextual LLM Chat! Type 'exit' to quit.")
    
    # Initialize conversation history
    history = []
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
            
        # TODO: Get response using conversation history
        # Update history
        # Print the response
        
def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM using conversation history")
    # TODO: Add arguments to the parser
    
    args = parser.parse_args()
    
    # TODO: Run the chat function with parsed arguments
    
if __name__ == "__main__":
    main()

In [ ]:
%%writefile utils/conversation.py
# utils/conversation.py

import requests
import argparse
import os

def get_response(prompt, history=None, model_name="google/flan-t5-base", api_key=None, history_length=3):
    """
    Get a response from the model using conversation history
    
    Args:
        prompt: The current user prompt
        history: List of previous (prompt, response) tuples
        model_name: Name of the model to use
        api_key: API key for authentication
        history_length: Number of previous exchanges to include in context
        
    Returns:
        The model's response
    """
    # TODO: Implement the contextual response function
    # Initialize history if None
    if history is None:
        history = []
        
    # TODO: Format a prompt that includes previous exchanges
    # Get a response from the API
    # Return the response
    pass

def run_chat():
    """Run an interactive chat session with context"""
    print("Welcome to the Contextual LLM Chat! Type 'exit' to quit.")
    
    # Initialize conversation history
    history = []
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
            
        # TODO: Get response using conversation history
        # Update history
        # Print the response
        
def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM using conversation history")
    # TODO: Add arguments to the parser
    
    args = parser.parse_args()
    
    # TODO: Run the chat function with parsed arguments
    
if __name__ == "__main__":
    main()

## 3. Testing and Evaluation

Create a script to test your chat implementations with specific healthcare questions.

In [ ]:
%%writefile utils/test_chat.py
# utils/test_chat.py

import os
import csv
from pathlib import Path

# Import our chat modules - since we're in the same directory
from one_off_chat import get_response as get_one_off_response
# Optionally import the conversation module if testing that too
# from conversation import get_response as get_contextual_response

def test_chat(questions, model_name="google/flan-t5-base", api_key=None):
    """
    Test the chat function with a list of questions
    
    Args:
        questions: A list of questions to test
        model_name: Name of the model to use
        api_key: API key for authentication
        
    Returns:
        A dictionary mapping questions to responses
    """
    results = {}
    
    for question in questions:
        print(f"Testing question: {question}")
        # Get response using the one-off chat function
        response = get_one_off_response(question, model_name, api_key)
        results[question] = response
        
    return results

def save_results(results, output_file="results/part_2/example.txt"):
    """
    Save the test results to a file
    
    Args:
        results: Dictionary mapping questions to responses
        output_file: Path to the output file
    """
    with open(output_file, 'w') as f:
        # Write header
        f.write("# LLM Chat Tool Test Results\n\n")
        
        # Write usage examples
        f.write("## Usage Examples\n\n")
        f.write("```bash\n")
        f.write("# Run the one-off chat\n")
        f.write("python utils/one_off_chat.py\n\n")
        f.write("# Run the contextual chat\n")
        f.write("python utils/conversation.py\n")
        f.write("```\n\n")
        
        # Write test results
        f.write("## Test Results\n\n")
        f.write("```csv\n")
        f.write("question,response\n")
        
        for question, response in results.items():
            # Format the question and response for CSV
            q = question.replace(',', '').replace('\n', ' ')
            r = response.replace(',', '').replace('\n', ' ')
            f.write(f"{q},{r}\n")
            
        f.write("```\n")

if __name__ == "__main__":
    # List of healthcare questions to test
    test_questions = [
        "What are the symptoms of gout?",
        "How is gout diagnosed?",
        "What treatments are available for gout?",
        "What lifestyle changes can help manage gout?",
        "What foods should be avoided with gout?"
    ]
    
    results = test_chat(test_questions)
    save_results(results)
    print("Test results saved to results/part_2/example.txt")

In [ ]:
# List of healthcare questions to test
test_questions = [
    "What are the symptoms of gout?",
    "How is gout diagnosed?",
    "What treatments are available for gout?",
    "What lifestyle changes can help manage gout?",
    "What foods should be avoided with gout?"
]

# Display the test questions
print("Test questions:")
for i, question in enumerate(test_questions, 1):
    print(f"{i}. {question}")

## Progress Checkpoints

1. **API Connection**:
   - [ ] Successfully connect to the Hugging Face API
   - [ ] Send a query and receive a response
   - [ ] Handle API errors gracefully

2. **Chat Function Implementation**:
   - [ ] Implement the get_response function
   - [ ] Create the run_chat function for interactive sessions
   - [ ] Handle errors and edge cases

3. **Command Line Interface**:
   - [ ] Create a parser with appropriate arguments
   - [ ] Implement the main function
   - [ ] Test the CLI functionality

4. **Testing and Evaluation**:
   - [ ] Test the functions with healthcare questions
   - [ ] Save the results in a structured format
   - [ ] Analyze the quality of responses

## Common Issues and Solutions

1. **API Access Issues**:
   - Problem: Rate limiting
   - Solution: Implement exponential backoff and retry logic
   - Problem: Authentication errors
   - Solution: Verify API key and environment variables

2. **Response Parsing Issues**:
   - Problem: Unexpected response format
   - Solution: Add error handling for different response structures
   - Problem: Empty or error responses
   - Solution: Provide meaningful fallback responses

3. **CLI Issues**:
   - Problem: Arguments not parsed correctly
   - Solution: Test with different argument combinations
   - Problem: Script not executable
   - Solution: Check file permissions

## What to Submit

1. Your implementation of the chat scripts:
   - Basic requirement: `utils/one_off_chat.py` for single prompt/response chat
   - Stretch goal (optional): `utils/conversation.py` for contextual chat
   - Testing script: `utils/test_chat.py` to evaluate your implementation

2. Test results in `results/part_2/example.txt` with the following format:
   - Usage examples section showing how to run your scripts
   - Test results section with CSV-formatted question/response pairs
   - If you implemented the stretch goal, include examples of contextual exchanges

The auto-grader should check:
1. That your chat scripts can be executed
2. That they correctly handle the test questions
3. That your results file contains the required sections

In [ ]:
# Run this cell to test your implementation
# Make sure to complete the TODO items in the scripts first!

# Run the test script (uncomment when your implementation is ready)
# %run utils/test_chat.py